In [1]:
import torch
from d2l import torch as d2l
from torch import nn

def corr2d(X,K):#定义二维互相关运算,K为卷积核，X为输入值
    h,w=K.shape
    Y = torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))#定义输出矩阵
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]=(X[i:i+h,j:j+w]*K).sum()
    return Y

class Conv2D(nn.Module):#卷积层定义函数
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))#可训练参数
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

X = torch.ones((6, 8))
X[:, 2:6] = 0
X

K = torch.tensor([[1.0, -1.0]])

Y = corr2d(X, K)
Y

corr2d(X.t(), K)


# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

conv2d.weight.data.reshape((1, 2))

epoch 2, loss 10.036
epoch 4, loss 2.193
epoch 6, loss 0.577
epoch 8, loss 0.182
epoch 10, loss 0.066


tensor([[ 0.9602, -1.0105]])